In [4]:
from tools import *
from config import *

In [5]:
kdays = 7
scale = 60

exp = f'final/firepoint/2x2/train/default/check_z-score/full_{kdays}_{scale}_700/baseline/'
features = read_object('relevant_features_risk.pkl', path=Path(exp))
features_name, _ = get_features_name_list(scale, train_features)
if kdays > 0:
    for k in range(1, kdays + 1):
            new_fet = [v+'_'+str(k) for v in varying_time_variables]
            train_features += [nf for nf in new_fet if nf.split('_')[0] in train_features]
            features_name, newShape = get_features_name_list(scale, train_features)
log_features(features, features_name)

2024-08-01 11:29:16,959 [INFO ]  (352, 'dayofyear')
2024-08-01 11:29:16,967 [INFO ]  (60, 'nesterov_sum')
2024-08-01 11:29:16,968 [INFO ]  (90, 'fwi_sum')
2024-08-01 11:29:16,970 [INFO ]  (93, 'dailySeverityRating_max')
2024-08-01 11:29:16,976 [INFO ]  (91, 'dailySeverityRating_mean')
2024-08-01 11:29:16,978 [INFO ]  (370, 'pastinfluence_sum')
2024-08-01 11:29:16,979 [INFO ]  (406, 'calendar_min_mean_1')
2024-08-01 11:29:16,981 [INFO ]  (387, 'fwi_mean_1')
2024-08-01 11:29:16,982 [INFO ]  (368, 'pastinfluence_max')
2024-08-01 11:29:16,983 [INFO ]  (55, 'dmc_sum')
2024-08-01 11:29:16,985 [INFO ]  (50, 'ffmc_sum')
2024-08-01 11:29:16,986 [INFO ]  (86, 'fwi_mean')
2024-08-01 11:29:16,987 [INFO ]  (431, 'dayofyear_min_1')
2024-08-01 11:29:16,988 [INFO ]  (45, 'dc_sum')
2024-08-01 11:29:16,989 [INFO ]  (48, 'ffmc_max')
2024-08-01 11:29:16,993 [INFO ]  (56, 'nesterov_mean')
2024-08-01 11:29:16,995 [INFO ]  (384, 'isi_mean_1')
2024-08-01 11:29:16,997 [INFO ]  (1458, 'dc_mean_6')
2024-08-01 11

In [3]:
features_name

{'temp': 6,
 'dwpt': 10,
 'rhum': 14,
 'prcp': 18,
 'wdir': 22,
 'wspd': 26,
 'prec24h': 30,
 'dc': 34,
 'ffmc': 38,
 'dmc': 42,
 'nesterov': 46,
 'munger': 50,
 'kbdi': 54,
 'isi': 58,
 'angstroem': 62,
 'bui': 66,
 'fwi': 70,
 'dailySeverityRating': 74,
 'temp16': 78,
 'dwpt16': 82,
 'rhum16': 86,
 'prcp16': 90,
 'wdir16': 94,
 'wspd16': 98,
 'prec24h16': 102,
 'days_since_rain': 106,
 'sum_consecutive_rainfall': 110,
 'sum_last_7_days': 114,
 'elevation': 118,
 'population': 122,
 'sentinel': 126,
 'landcover': 146,
 'foret': 154,
 'highway': 230,
 'dynamicWorld': 254,
 'Calendar': 290,
 'Historical': 305,
 'Geo': 309,
 'AutoRegressionBin': 310}

In [4]:
features_name['dynamicWorld'] + (dynamic_world_variables.index('grass') * 4)

262